# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [2]:
# Note that this is not the same db we have used in course videos, please download from this link
# https://drive.google.com/file/d/1O-1-L1DdNxEK6O6nG2jS31MbrMh-OnXM/view?usp=sharing

In [5]:
conn = sqlite3.connect("./Db-IMDB-Assignment.db")
cursor = conn.cursor()

#### Overview of all tables

In [6]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [7]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

## Explanation for Answer-1

This query is selecting data from multiple tables (Person, M_Director, Movie, M_Genre, and Genre) and combining them using JOINs and subqueries.

The first subquery, "leap_years", selects the MID (movie ID) from the Movie table where the year is a leap year (divisible by 4 and not divisible by 100, or divisible by 400).

The second subquery, "comedy_movies", selects the MID from the M_Genre table where the GID (genre ID) is in the Genre table and the Name column contains the word "comedy".

The main query then selects the Name, title, and year from the Person table, joining it with the M_Director table on PID and the Movie table on MID. Then it filters the results to only include movies where the MID is in both the leap_years subquery and the comedy_movies subquery, so it returns the movies that are both leap year and comedy.

In [8]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    assert (q1_results.shape == (232,3))

query1 = """
WITH leap_years AS (
    SELECT MID
    FROM Movie
    WHERE (CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) % 4 = 0 and CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) % 100 <> 0) or
      (CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER) % 400 = 0)
),
comedy_movies AS (
    SELECT MID
    FROM M_Genre
    WHERE GID IN (
        SELECT GID
        FROM Genre
        WHERE Name LIKE '%comedy%'
    )
)
SELECT p.Name, m.title, m.year
FROM Person p
JOIN M_Director md ON p.PID = md.PID
JOIN Movie m ON m.MID = md.MID
WHERE m.MID IN (SELECT * FROM leap_years) AND m.MID IN (SELECT * FROM comedy_movies)"""
grader_1(query1)

                Name                              title  year
0       Milap Zaveri                         Mastizaade  2016
1       Danny Leiner  Harold & Kumar Go to White Castle  2004
2     Anurag Kashyap                 Gangs of Wasseypur  2012
3       Frank Coraci        Around the World in 80 Days  2004
4      Griffin Dunne             The Accidental Husband  2008
5        Anurag Basu                             Barfi!  2012
6    Gurinder Chadha                  Bride & Prejudice  2004
7         Mike Judge    Beavis and Butt-Head Do America  1996
8   Tarun Mansukhani                            Dostana  2008
9       Shakun Batra                      Kapoor & Sons  2016
CPU times: user 292 ms, sys: 3.25 ms, total: 295 ms
Wall time: 294 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

## Explanation for Answer-2 

## Explanation

This query is selecting data from two tables (Movie and Person) and combining them using JOIN and subquery.

The subquery, "Anand", selects the title of the movie and the PID (person ID) from the M_CAST table, joining it with the Movie table on MID. Then it filters the results to only include the rows where the title of the movie is 'Anand'.

The main query then selects the distinct and trimmed names of the actors from the Person table, joining it with the subquery "Anand" on the trimmed PIDs. The result of this query will be a list of distinct actors names who acted in the movie 'Anand'.

In [9]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))


query2 = """
WITH Anand AS (
    SELECT M.title AS Movie, MC.PID
    FROM MOVIE AS M
    JOIN M_CAST AS MC ON M.MID = MC.MID
    WHERE M.title = 'Anand'
)

SELECT DISTINCT TRIM(P.NAME) AS Actors
FROM Anand
JOIN PERSON AS P ON TRIM(P.PID) = TRIM(Anand.PID);
"""
grader_2(query2)

             Actors
0  Amitabh Bachchan
1     Rajesh Khanna
2     Sumita Sanyal
3        Ramesh Deo
4         Seema Deo
5    Asit Kumar Sen
6        Dev Kishan
7      Atam Prakash
8     Lalita Kumari
9            Savita
CPU times: user 109 ms, sys: 4.41 ms, total: 113 ms
Wall time: 112 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [10]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 =""" 
Select p.PID from Person p 
inner join 
(
    select trim(mc.PID) PD, mc.MID from M_cast mc 
where mc.MID 
in 
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)<1970
)
) r1 
on r1.PD=p.PID 
"""
query_more_1990 =""" 
Select p.PID from Person p 
inner join 
(
    select trim(mc.PID) PD, mc.MID from M_cast mc 
where mc.MID 
in 
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)>1990
)
) r1 
on r1.PD=p.PID """
print(grader_3a(query_less_1970, query_more_1990))

# using the above two queries, you can find the answer to the given question 

(4942, 1)
(62570, 1)
True
CPU times: user 100 ms, sys: 5.29 ms, total: 105 ms
Wall time: 104 ms


## Explanation for Answer-3

This query is selecting data from three tables (Movie, M_cast, and Person) and combining them using JOIN and subqueries.

The first subquery "actors_before_1970" selects the distinct and trimmed PID (person ID) of actors who have acted in movies before 1970. It does this by joining the movie table with the M_cast table on MID, and then filtering for only the movies with a year less than 1970.

The second subquery "actors_after_1990" selects the distinct and trimmed PID of actors who have acted in movies after 1990. It does this by joining the movie table with the M_cast table on MID, and then filtering for only the movies with a year greater than 1990.

The main query then selects the distinct and trimmed names of the actors who have acted in movies both before 1970 and after 1990. It does this by joining the subquery "actors_before_1970" with the subquery "actors_after_1990" on PID, and then joining with the Person table on trimmed PID. The result of this query will be a list of actors who have acted in movies both before 1970 and after 1990.

In [11]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 = """
WITH actors_before_1970 AS (
SELECT DISTINCT TRIM(mc.PID) AS pid
FROM movie m
JOIN m_cast mc ON m.MID = mc.MID
WHERE CAST(SUBSTR(m.year, -4) AS INT) < 1970
),
actors_after_1990 AS (
SELECT DISTINCT TRIM(mc.PID) AS pid
FROM movie m
JOIN m_cast mc ON m.MID = mc.MID
WHERE CAST(SUBSTR(m.year, -4) AS INT) > 1990
)
SELECT DISTINCT TRIM(p.NAME) AS actors_before_1970_after_1990
FROM actors_before_1970 a_1970
JOIN actors_after_1990 a_1990 ON a_1970.pid = a_1990.pid
JOIN person p ON a_1970.pid = TRIM(p.PID); 
"""
grader_3(query3)

  actors_before_1970_after_1990
0                  Rishi Kapoor
1              Amitabh Bachchan
2                        Asrani
3                  Zohra Sehgal
4               Parikshat Sahni
5                 Rakesh Sharma
6                   Sanjay Dutt
7                     Ric Young
8                         Yusuf
9                Suhasini Mulay
CPU times: user 141 ms, sys: 4.7 ms, total: 145 ms
Wall time: 144 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [12]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

query_4a ="""
SELECT trim(p.name) AS directors, COUNT(DISTINCT md.mid) AS movie_count
FROM person p
JOIN m_director md ON trim(p.pid) = trim(md.pid)
GROUP BY directors;
"""
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

                  directors  movie_count
0              A. Bhimsingh            7
1            A. Jagannathan            2
2                  A. Majid            1
3                  A. Muthu            1
4             A.M.R. Ramesh            1
5            A.P. Nagarajan            1
6           A.R. Murugadoss            6
7  A.S. Ravi Kumar Chowdary            2
8              Aamir Bashir            1
9                Aamir Khan            1
False
CPU times: user 8.49 s, sys: 0 ns, total: 8.49 s
Wall time: 8.49 s


## Explanation for Answer-4

This subquery is selecting data from two tables (Person and M_director) and using JOIN to combine them based on the person ID (PID). It then performs several operations on the data:

    It selects the trimmed name of the person (director) and the count of distinct movies they directed.
    It groups the results by the directors' names.
    It filters the results to only include directors who have directed at least 10 movies.
    It orders the results in descending order by the number of movies directed.

So the final output is a list of directors with their directed movie counts which are 10 or more, ordered by movie count in descending order.

In [13]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 = """
SELECT trim(p.name) AS directors, COUNT(DISTINCT md.mid) AS movie_count
FROM person p
JOIN m_director md ON trim(p.pid) = trim(md.pid)
GROUP BY directors
HAVING movie_count >= 10
ORDER BY movie_count DESC;"""
grader_4(query4)

              directors  movie_count
0          David Dhawan           39
1          Mahesh Bhatt           36
2       Ram Gopal Varma           30
3          Priyadarshan           30
4          Vikram Bhatt           29
5  Hrishikesh Mukherjee           27
6           Yash Chopra           21
7        Shakti Samanta           19
8       Basu Chatterjee           19
9          Subhash Ghai           18
CPU times: user 8.13 s, sys: 0 ns, total: 8.13 s
Wall time: 8.13 s


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [14]:
cursor.executescript("""
create index idx_person_pid on person(trim(pid));
create index idx_m_cast_pid on m_cast(trim(pid));
create index idx_person_gender on person(lower(trim(gender)));
""")

In [17]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10)) 
    return (query_5aa.shape == (8846,3)) # this is incorrect because total number of movies are only 3473 and number of actors are 37566

query_5aa = """
select mc.mid mid, p.gender gender, count(gender) count
from person p
join m_cast mc
on trim(p.pid) = trim(mc.pid)
group by gender, mid
order by count desc;
"""

print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10)) 
    return (query_5ab.shape == (3469, 3))

query_5ab ="""
    select mc.mid mid, p.gender gender, count(gender) count
    from person p
    join m_cast mc
    on trim(p.pid) = trim(mc.pid)
    group by gender, mid
    having gender = 'Male'
    order by count
    desc;
"""

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question

         mid  gender  count
0  tt0451631    Male    199
1  tt6173990    Male    173
2  tt1573482    Male    150
3  tt3498820    Male    135
4  tt1981128    Male    120
5  tt1188996    Male    118
6  tt5164214    Male    110
7  tt5164214  Female    107
8  tt6926486    Male    103
9  tt1190080    Male    102
True
         mid gender  count
0  tt0451631   Male    199
1  tt6173990   Male    173
2  tt1573482   Male    150
3  tt3498820   Male    135
4  tt1981128   Male    120
5  tt1188996   Male    118
6  tt5164214   Male    110
7  tt6926486   Male    103
8  tt1190080   Male    102
9  tt3863552   Male    101
True
CPU times: user 155 ms, sys: 32 ms, total: 187 ms
Wall time: 186 ms


## Explanation for Answer-5a

This query is using Common Table Expressions (CTEs) to first select movies that have at least one male cast member, and then select movies that do not have any male cast members.

    The first CTE, "male_movies," selects the Movie ID (MID) from the M_CAST table and joins it with the Person table on the Person ID (PID) of the cast member. It filters the results to include only movies where the cast member is male or gender is not specified.

    The second CTE, "female_movies," selects the Movie ID (MID) and year from the Movie table and filters the results to include only movies that are not in the "male_movies" CTE, effectively selecting movies that have no male cast members.

    The final query then selects the year and count of distinct female-only movies for each year and groups by year. The result is ordered by year.

So the final output is a list of years and the number of movies that have only female cast members which are produced in each year.

In [20]:
cursor.executescript("""
CREATE INDEX idx_m_cast_mid ON m_cast (mid);
CREATE INDEX idx_movie_mid ON movie (mid);
CREATE INDEX idx_movie_year ON movie (year);
""")

In [21]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    assert (q5a_results.shape == (4,2))

query5a = """
with only_female_movies as (
	select 
		distinct trim(mc.mid) mid, 
		sum(case when lower(trim(p.gender)) in ('male', 'none') then 1 else 0 end) males,
		sum(case when lower(trim(p.gender)) = 'female' then 1 else 0 end) females
	from person p
	join m_cast mc
	on trim(p.pid) = trim(mc.pid)
	group by mid
	having males = 0
)

select substr(year, -4) year, count(distinct(trim(ofm.mid))) female_movies
from Movie m
join only_female_movies ofm
on trim(m.mid) = ofm.mid
group by substr(year, -4);
"""
grader_5a(query5a)

   year  female_movies
0  1939              1
1  1999              1
2  2000              1
3  2018              1
CPU times: user 71.4 ms, sys: 3.95 ms, total: 75.3 ms
Wall time: 74 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

## Explanation for Answer-5b

This query uses a combination of subqueries and joins to determine the percentage of movies that were directed, written, or produced by women in a given year.

The first subquery, "only_female_movies," selects distinct movies, and counts the number of male and female cast members for each movie. It then filters out any movies that have any male cast members by using the HAVING clause on the count of males.

The second subquery, "total_movies_year," selects the year and the total number of movies for each year.

The third subquery, "female_movies_year," selects the year and the total number of movies for each year where only females casted in the movie.

In the final select statement, the query joins the subqueries on the year column and calculates the percentage of movies with only female cast members by dividing the number of movies with only female cast members by the total number of movies for that year. The results are ordered by year.

In [26]:
cursor.executescript("""
CREATE INDEX idx_mid_mc ON m_cast (trim(mid));
CREATE INDEX idx_pid_p ON person (trim(pid));
CREATE INDEX idx_gender_p ON person (lower(trim(gender)));
CREATE INDEX idx_year_m ON movie (substr(year, -4));
""")

In [27]:
%%timeit
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

query5b = """
with only_female_movies as (
	select 
		distinct trim(mc.mid) mid, 
		sum(case when lower(trim(p.gender)) in ('male', 'none') then 1 else 0 end) males,
		sum(case when lower(trim(p.gender)) = 'female' then 1 else 0 end) females
	from person p
	join m_cast mc
	on trim(p.pid) = trim(mc.pid)
	group by mid
	having males = 0
),

total_movies_year as (
	select distinct substr(year, -4) year, count(distinct(trim(mid))) total_movies
	from movie
	group by substr(year, -4)
),

female_movies_year as (
    select substr(year, -4) year, count(distinct(trim(ofm.mid))) female_movies
    from Movie m
    join only_female_movies ofm
    on trim(m.mid) = ofm.mid
	group by substr(year, -4)
)

select year, (cast(female_movies as float) * 100 / cast(total_movies as float)) female_percentage, total_movies
from total_movies_year
join female_movies_year
using (year)
order by year;
"""
grader_5b(query5b)

   year  female_percentage  total_movies
0  1939          50.000000             2
1  1999           1.515152            66
2  2000           1.562500            64
3  2018           0.961538           104
   year  female_percentage  total_movies
0  1939          50.000000             2
1  1999           1.515152            66
2  2000           1.562500            64
3  2018           0.961538           104
   year  female_percentage  total_movies
0  1939          50.000000             2
1  1999           1.515152            66
2  2000           1.562500            64
3  2018           0.961538           104
   year  female_percentage  total_movies
0  1939          50.000000             2
1  1999           1.515152            66
2  2000           1.562500            64
3  2018           0.961538           104
   year  female_percentage  total_movies
0  1939          50.000000             2
1  1999           1.515152            66
2  2000           1.562500            64
3  2018         

   year  female_percentage  total_movies
0  1939          50.000000             2
1  1999           1.515152            66
2  2000           1.562500            64
3  2018           0.961538           104
   year  female_percentage  total_movies
0  1939          50.000000             2
1  1999           1.515152            66
2  2000           1.562500            64
3  2018           0.961538           104
   year  female_percentage  total_movies
0  1939          50.000000             2
1  1999           1.515152            66
2  2000           1.562500            64
3  2018           0.961538           104
   year  female_percentage  total_movies
0  1939          50.000000             2
1  1999           1.515152            66
2  2000           1.562500            64
3  2018           0.961538           104
   year  female_percentage  total_movies
0  1939          50.000000             2
1  1999           1.515152            66
2  2000           1.562500            64
3  2018         

## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

## Explanation for Answer-6

This query is using a common table expression (CTE) called "movie_cast_count" to first count the distinct number of actors (or "cast members") in each movie by joining the "M_CAST" table to itself and grouping by movie ID (MID).

The CTE is then joined to the "MOVIE" table, using the MID to match the movie data with the cast count data, and the results are returned in a query with columns "movie" and "cast_count", ordered by cast_count in descending order.

In [ ]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 = """
WITH
    movie_cast_count AS (
        SELECT COUNT(DISTINCT TRIM(MC.PID)) cast_count, MC.MID mid
        FROM M_CAST MC
        GROUP BY MC.MID
    )

SELECT M.title movie, C.cast_count
FROM MOVIE M
JOIN movie_cast_count C ON C.mid = M.MID
ORDER BY cast_count DESC
"""
grader_6(query6)

### Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D

In [ ]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

query7a = """
select year, count(MID) total_movies
from movie
group by substr(year, -4)
order by total_movies;
"""
grader_7a(query7a)

# using the above query, you can write the answer to the given question

In [ ]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

query7b = """   
with movie_table as (
	select cast(substr(trim(year), -4) as INTEGER) year, count(MID) total_movies
	from movie
	group by year
	order by total_movies
)

select t1.year year_1, t2.year year_2, t1.total_movies tm_1, t2.total_movies tm_2
from movie_table t1, movie_table t2
where year_2 <= (year_1 + 9) and year_2 >= year_1
order by year_1, year_2;
          """
#grader_7b(query7b) -> I am using another approach to solve the problem
# if you see the below results the first movie year is less than 2nd movie year and 
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

## Explanation for Answer-7

This query calculates the number of movies produced in each decade, and returns the decade with the most movies.

First, it creates a subquery "decades" that selects the distinct decades (based on the year of the movie) from the "Movie" table, and also creates 2 columns, "decade_start" and "decade_end" which are decade and decade+9 respectively. It then orders them in descending order.

Then, it creates another subquery "movies_in_year" that selects the total number of distinct movies (based on their ID) in each year and groups them by year.

Finally, it selects the sum of total movies produced in each decade and decade, by joining the 2 subqueries "movies_in_year" and "decades" on the year being between the decade_start and decade_end. It then groups the results by decade and orders them by the sum of total movies produced in each decade in descending order and limits the result to the top 1.

In [ ]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 = """
with decades as (
	select distinct cast(substr(year, -4) as INTEGER) decade, cast(substr(year, -4) as INTEGER) decade_start, (cast(substr(year, -4) as INTEGER) + 9) decade_end
	from Movie
	order by decade_start desc
),

movies_in_year as (
	select count(distinct mid) total_movies, cast(substr(year, -4) as INTEGER) year
	from movie
	group by year
)
select sum(total_movies) total_movies_decade, d.decade
from movies_in_year miy
join decades d
on miy.year between d.decade_start and d.decade_end
group by d.decade
order by total_movies_decade
desc
limit 1;
"""
grader_7(query7)
# if you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [ ]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

query8a = """
SELECT TRIM(MC.PID) ACTOR_PID, TRIM(MD.PID) DIRECTOR_PID, COUNT(DISTINCT TRIM(MD.MID)) NUM_MOVIES_TOGETHER
FROM M_CAST MC
JOIN M_DIRECTOR MD
ON TRIM(MC.MID) = TRIM(MD.MID)
GROUP BY ACTOR_PID, DIRECTOR_PID;
"""
grader_8a(query8a)

# using the above query, you can write the answer to the given question

## Explanation for Answer-8

This query is attempting to find the actors who have worked with the director Yash Chopra the most, compared to the actors' maximum number of movies worked with any other director.

The query starts by creating a subquery called YC, which finds the person ID (DID) of the director Yash Chopra.

Next, it creates a subquery called ADM, which finds the actor ID (AID), director ID (DID), and number of common movies between the actor and director. This subquery joins the M_CAST and M_DIRECTOR table on the movie ID and groups the result by AID and DID.

Then, it creates a subquery called YCA, which finds the distinct actor IDs that have worked with Yash Chopra (DID is the one found in the YC subquery).

The next subquery, YCA_YM, finds the number of movies the actors in YCA have worked with Yash Chopra, while the subquery YCA_MM finds the maximum number of movies the actors in YCA have worked with any other director.

Finally, the subquery YCA_MMM finds the actors from YCA_YM whose proportion of movies worked with Yash Chopra is greater than or equal to 1.

The final SELECT statement joins the Person table with YCA_MMM on the actor ID, so that it can retrieve the name of the actors and the actor ID. The result is the actors who have worked the most with Yash Chopra compared to the actors' maximum number of movies worked with any other director.

In [ ]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = """
WITH
YC AS (
	SELECT TRIM(P.PID) DID
	FROM PERSON P
	WHERE TRIM(P.NAME) LIKE "%Yash Chopra%"
),

ADM AS (
	SELECT TRIM(MC.PID) AID, TRIM(MD.PID) DID, COUNT(DISTINCT TRIM(MD.MID)) COMMON_MOVIES
	FROM M_CAST MC
	JOIN M_DIRECTOR MD
	ON TRIM(MC.MID) = TRIM(MD.MID)
	GROUP BY AID, DID
),

YCA AS (
	SELECT DISTINCT AID
	FROM ADM
	WHERE DID = (SELECT DID FROM YC)
),

YCA_YM AS (
	SELECT AID, COMMON_MOVIES YASH_MOVIES
	FROM ADM
	WHERE DID = (SELECT DID FROM YC)
	AND AID IN (SELECT AID FROM YCA)
),

YCA_MM AS (
	SELECT AID, MAX(COMMON_MOVIES) MAX_MOVIES
	FROM ADM
	WHERE AID IN (SELECT AID FROM YCA)
	GROUP BY AID
	ORDER BY MAX_MOVIES
),

YCA_MMM AS (
	SELECT AID
	FROM YCA_YM
	JOIN YCA_MM
	USING (AID)
	WHERE YASH_MOVIES / MAX_MOVIES >= 1
)

SELECT NAME, AID
FROM Person P
JOIN YCA_MMM
ON YCA_MMM.AID = TRIM(P.PID);
"""
grader_8(query8)

## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [ ]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """
WITH 

SRK_PID AS (
	SELECT TRIM(P.PID) PID
	FROM Person P
	WHERE lower(trim(P.Name)) = "shah rukh khan"
),

SRK_MOVIES AS (
	SELECT DISTINCT TRIM(MC.MID) MID, SRK_PID.PID
	FROM M_Cast MC, SRK_PID
	WHERE TRIM(MC.PID) = SRK_PID.PID
),

SRK_COACTORS_PID AS (
	SELECT DISTINCT TRIM(MC.PID) PID
	FROM M_Cast MC, SRK_MOVIES SM
	WHERE trim(MC.MID) = SM.MID and TRIM(MC.PID) != SM.PID
)

SELECT distinct PID
FROM SRK_COACTORS_PID;
"""
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

## Explanation for Answer-9

The query uses common table expressions (CTEs) to break down the problem into smaller steps.

    The first CTE, "SRK", selects the PID (person ID) of Shah Rukh Khan from the Person table.

    The second CTE, "SRK_MOVIES", selects the MID (movie ID) of all movies Shah Rukh Khan has acted in by joining the M_Cast table with the SRK CTE.

    The third CTE, "SRK_COACTORS", selects the PID of all actors who have worked with Shah Rukh Khan by joining the M_Cast table with the SRK_MOVIES CTE and SRK CTE, but only distinct PID.

    The fourth CTE, "SRK_COACTORS_ONLY_MOVIES", selects the MID of all movies where actors who have worked with Shah Rukh Khan are acting but Shah Rukh Khan is not by joining the M_Cast table with the SRK_COACTORS CTE and SRK_MOVIES CTE, but only distinct MID.

    The final SELECT statement selects the PID of all actors who have worked in movies where actors who have worked with Shah Rukh Khan are acting, but not Shah Rukh Khan himself. This is done by joining the M_Cast table with the SRK_COACTORS_ONLY_MOVIES CTE and SRK_COACTORS CTE, and only selecting the PIDs that are not already in the SRK_COACTORS CTE.

In [ ]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = """
WITH

SRK AS (
	SELECT trim(PID) PID
	FROM Person
	WHERE trim(lower(Name)) like "shah rukh khan"
),

SRK_MOVIES AS (
	SELECT DISTINCT trim(MC.MID) MID
	FROM M_Cast MC
	JOIN SRK
	ON trim(MC.PID) = SRK.PID
),

SRK_COACTORS AS (
	SELECT distinct trim(MC.PID) PID
	FROM M_Cast MC
	JOIN SRK_MOVIES SM
	ON trim(MC.MID) = SM.MID
	JOIN SRK 
	ON trim(MC.PID) != SRK.PID
),

SRK_COACTORS_ONLY_MOVIES AS (
	SELECT DISTINCT trim(MC.MID) MID
	FROM M_Cast MC
	JOIN SRK_COACTORS SC
	ON trim(MC.PID) = SC.PID
	WHERE trim(MC.MID) NOT IN (SELECT MID FROM SRK_MOVIES)
)

SELECT DISTINCT trim(MC.PID) PID
FROM M_Cast MC
JOIN SRK_COACTORS_ONLY_MOVIES SCM
ON trim(MC.MID) = SCM.MID
WHERE trim(MC.PID) NOT IN (SELECT PID FROM SRK_COACTORS);
"""
grader_9(query9)